### <b>Import the Libraries</b>

In [1]:
import json
import re
import pandas as pd
import datetime
from twython import Twython
from textblob import TextBlob
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

### <b>Create function to get the top N cryptocurrencies on coinmarketcap.com</b>

In [2]:
def getTopNCryptocurrencies(N):
    """
    Gets the top N cryptocurrencies in coinmarketcap.com
    """
    
    #Limit returns the top N cryptocurrencies on coinmarketcap
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':f'{N}',
      'convert':'USD'
    }
    
    #Provide your API key from coinmarketcap
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': 'GET FROM COINMARKETCAP',
    }
    
    #Start session to connect to coinmarketcap
    session = Session()
    session.headers.update(headers)
    
    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    #Append the top N coins and return them as a list
    coins = []
    for i in range(N):
        coins.append((data['data'][i]['name'], data['data'][i]['symbol'], data['data'][i]['slug']))
                     
    return coins

### <b>Retrieve the top N coins</b>

In [3]:
n = 10

print(f"Top {n} cryptocurrencies today:")
for coin in getTopNCryptocurrencies(n):
    print(coin[0], '/', coin[1], '/', coin[2])

Top 10 cryptocurrencies today:
Bitcoin / BTC / bitcoin
Ethereum / ETH / ethereum
Binance Coin / BNB / binance-coin
Dogecoin / DOGE / dogecoin
Cardano / ADA / cardano
Tether / USDT / tether
XRP / XRP / xrp
Polkadot / DOT / polkadot-new
Internet Computer / ICP / internet-computer
Bitcoin Cash / BCH / bitcoin-cash


### <b>Store twitter credentials in json file</b>

In [4]:
def twitterCredentials(files):
    """
    Saves twitter credentials (API key, token, etc.)
    """
    
    credentials = {}
    credentials['API_KEY'] = 'GET FROM TWITTER'
    credentials['API_SECRET_KEY'] = 'GET FROM TWITTER'
    credentials['ACCESS_TOKEN'] = 'GET FROM TWITTER'
    credentials['ACCESS_SECRET_TOKEN'] = 'GET FROM TWITTER'
    
    with open(files, "w") as file:
        json.dump(credentials, file)
        print("Credentials Saved")
        return

In [5]:
twitterCredentials("twitter_credentials.json")

Credentials Saved


### <b>Perform sentiment analysis using Twython and TextBlob libraries</b>

In [6]:
def twitterSentiment(cryptocurrency, language = 'en'):
    """
    Gets the sentiment of a coin through polarity and subjectivity
    """
    
    #This will open the credentials that we have saved earlier
    with open("twitter_credentials.json", "r") as file:
        credential = json.load(file)
        
    coin_tweets = Twython(credential['API_KEY'], credential['API_SECRET_KEY'])
    
    keywords = " / ".join(cryptocurrency)
    
    query = {'q': keywords,
             'result_type': 'popular',
             'lang': language}
    
    #Create a dictionary containing details about the cryptocurrency
    #Polarity is a value [-1, 1] denoting negative and positive sentiment
    #Subjectivity is a value [0, 1] denoting subjectivity of tweets
    my_dict = {'coin': [], 'date': [], 'text': [], 'polarity': [], 'subjectivity': []}
    
    for status in coin_tweets.search(**query)['statuses']:
        my_dict['coin'].append(keywords)
        my_dict['date'].append(status['created_at'])
        my_dict['text'].append(status['text'])
        
        tweet = status['text'].encode("utf-8")
        tweet = " ".join(re.findall("[a-zA-Z]+", str(tweet)))
        
        analysis = TextBlob(tweet.strip())
        sentiment = analysis.sentiment
        my_dict['polarity'] = sentiment.polarity
        my_dict['subjectivity'] = sentiment.subjectivity
        
    data = pd.DataFrame(my_dict)
        
    return (float(data.loc[:,'polarity'].mean()), float(data.loc[:,'subjectivity'].mean()))

### <b>Testing our cryptocurrency sentiment analysis script</b>

In [7]:
print('CRYPTOCURRENCY SENTIMENT ANALYSIS...')
print(datetime.datetime.today())

# create a list for statistics
cryptocurrencies = []

#Get top N cryptocurrencies
for position, cryptocurrency in enumerate(getTopNCryptocurrencies(n)):
    print(position+1, cryptocurrency)
    # get mean polarity and subjectivity for a crypto (using only 15 tweets)
    pol, subj = twitterSentiment(cryptocurrency,language='en')
    # add them to stats
    cryptocurrencies.append((cryptocurrency[0]+'/'+cryptocurrency[1], pol, subj))

# create a dataframe for the mean values using the above list
cryptocurrency_stats = pd.DataFrame(cryptocurrencies, columns=['Cryptocurrency','Polarity','Subjectivity'])

# Sort the results using polarity descending
cryptocurrency_stats.sort_values(by='Polarity', inplace=True, ascending=False)  

# print means of the evaluations
print(f'\nTOP {n} CRYPTOCURRENCY STATISTICS')
print(cryptocurrency_stats)
print('\nHIGHEST POLARITY:')
print(cryptocurrency_stats.loc[cryptocurrency_stats['Polarity'].idxmax()])
print('\nHIGHEST SUBJECTIVITY:')
print(cryptocurrency_stats.loc[cryptocurrency_stats['Subjectivity'].idxmax()])
print('\nANALYSIS DONE!')

CRYPTOCURRENCY SENTIMENT ANALYSIS...
2021-05-15 01:29:15.375829
1 ('Bitcoin', 'BTC', 'bitcoin')
2 ('Ethereum', 'ETH', 'ethereum')
3 ('Binance Coin', 'BNB', 'binance-coin')
4 ('Dogecoin', 'DOGE', 'dogecoin')
5 ('Cardano', 'ADA', 'cardano')
6 ('Tether', 'USDT', 'tether')
7 ('XRP', 'XRP', 'xrp')
8 ('Polkadot', 'DOT', 'polkadot-new')
9 ('Internet Computer', 'ICP', 'internet-computer')
10 ('Bitcoin Cash', 'BCH', 'bitcoin-cash')

TOP 10 CRYPTOCURRENCY STATISTICS
          Cryptocurrency  Polarity  Subjectivity
6                XRP/XRP  0.500000      0.500000
5            Tether/USDT  0.400000      0.400000
0            Bitcoin/BTC  0.366667      0.700000
1           Ethereum/ETH  0.125000      0.216667
3          Dogecoin/DOGE  0.000000      0.550000
4            Cardano/ADA  0.000000      0.000000
9       Bitcoin Cash/BCH -0.050000      0.350000
2       Binance Coin/BNB       NaN           NaN
7           Polkadot/DOT       NaN           NaN
8  Internet Computer/ICP       NaN           NaN
